### Introduction
Coming soon ...

In [254]:
# Import modules and packages
import copy
from pomegranate import *
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc, roc_auc_score,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support, precision_score, accuracy_score, matthews_corrcoef)

In [255]:
def read_model_from_json(filename=None, verbose=False):
    """
    Reads and returns model data from JSON file.
    
    Parameters
    ----------
    filename : str
               Path to a saved model in JSON format
    verbose  : bool, optional, default False
               When True, prints progress to STDOUT
    
    Returns
    ----------
    dict
    JSON data as dictionary
    """
    
    with open(filename, 'r') as f:
        modeldata = json.load(f)
        #print(modeldata['edges'][2])
        if verbose: print("Model read from:", filename)
        return(modeldata)



def train_hmm(distribution=None, n_components=None, state_names=None, X=None, labels=None, algorithm=None, n_jobs=None):
    """
    Helper function to train and bake Pomogranate HMM model. Returns baked Pomogranate HMM model.
    Make sure you import the pomogranate package as-
    from pomegranate import *
    
    This function uses (docstrings are appended below)-
    pomogranate.HiddenMarkovModel.from_samples
    pomogranate.HiddenMarkovModel.bake
    
    """
    
    """
    # Example:
    model = HiddenMarkovModel.from_samples(distribution=DiscreteDistribution, n_components=2,
                                         state_names=['F','T'],
                                         X=train_seqs, labels=train_labels,
                                         algorithm='labeled',
                                         n_jobs=2)
    """
    
    model = HiddenMarkovModel.from_samples(distribution=distribution, n_components=n_components,
                                         state_names=state_names,
                                         X=X, labels=labels,
                                         algorithm=algorithm,
                                         n_jobs=n_jobs)
    model.bake()
    return(model)
# Append docstring
train_hmm.__doc__ += "\nHiddenMarkovModel.from_samples\n\n" + HiddenMarkovModel.from_samples.__doc__
train_hmm.__doc__ += "\n\nHiddenMarkovModel.bake\n\n" + HiddenMarkovModel.bake.__doc__


In [256]:
def validate(model=None, states=None, true_seqs=None, true_labels=None):
    """
    Scan 
    """
    y_true_all = []
    y_pred_all = []
    F_predictedAs_T_fraction = []
    T_predictedAs_F_fraction = []
    recall = []
    precision = []
    accuracy = []
    f1 = []
    for i in true_seqs.index:
        y_true = [int(ch=='T') for ch in true_labels[i]]
        y_true_all += copy.copy(y_true)
        y_pred = model.predict(list(true_seqs[i]))
        y_pred_all += copy.copy(y_pred)
        #print([int(ch=='T') for ch in true_labels[i]])
        #print("".join([states[i] for i in pred_labels]))
        #print(pred_labels)
        #print(i,"\n",true_seqs[i],"\n",true_labels[i],"\n","".join([states[i] for i in y_pred]))

        """
        if 1:#i == ">tr|A0A0R2L6D1|tig":
            confmat = confusion_matrix(y_true,y_pred)
            print(confmat)
            print(recall_score(y_true,y_pred))
            print(precision_score(y_true,y_pred))
            print(accuracy_score(y_true,y_pred))
            #print(i, true_seqs[i], true_labels[i])
            print("---")
        """

        confmat = confusion_matrix(y_true,y_pred)
        #print(confmat)
        #print(confmat.shape)
        if confmat.shape[0] == 1 and confmat.shape[1] == 1:
            F_predictedAs_T = 0
            T_predictedAs_F = 0
        else:
            F_predictedAs_T = confmat[0,1]
            T_predictedAs_F = confmat[1,0]
            num_res = np.sum(confmat)
            F_predictedAs_T_fraction.append(F_predictedAs_T/num_res)
            T_predictedAs_F_fraction.append(T_predictedAs_F/num_res)
        recall.append(recall_score(y_true,y_pred))
        precision.append(precision_score(y_true,y_pred))
        accuracy.append(accuracy_score(y_true,y_pred))
        f1.append(f1_score(y_true,y_pred))

        #if accuracy_score(y_true,y_pred) <= 0.75:
        #    print(i,"\n",true_seqs[i],"\n",true_labels[i],"\n","".join([states[i] for i in y_pred]))
    

    #print(np.mean(recall), np.mean(precision), np.mean(accuracy), np.mean(f1), np.mean(F_predictedAs_T_fraction), np.mean(T_predictedAs_F_fraction))
    #print(np.std(recall),    np.std(precision),    np.std(accuracy), np.std(f1), np.std(F_predictedAs_T_fraction), np.std(T_predictedAs_F_fraction))

    """
    print("Histograms\n")
    print("Recall:")
    H,xe = histogram1d(data=recall,binsize=0.1,xmin=0.0,xmax=1.0)
    print(H,"\n",xe,"\n")
    print("Precision:")
    H,xe = histogram1d(data=precision,binsize=0.1,xmin=0.0,xmax=1.0)
    print(H,"\n",xe,"\n")
    print("Accuracy:")
    H,xe = histogram1d(data=accuracy,binsize=0.1,xmin=0.0,xmax=1.0)
    print(H,"\n",xe,"\n")
    print("F1 score:")
    H,xe = histogram1d(data=f1,binsize=0.1,xmin=0.0,xmax=1.0)
    print(H,"\n",xe,"\n")
    print("F_predictedAs_T_fraction:")
    H,xe = histogram1d(data=F_predictedAs_T_fraction,binsize=0.1,xmin=0.0,xmax=1.0)
    print(H,"\n",xe,"\n")
    print("T_predictedAs_F_fraction:")
    H,xe = histogram1d(data=T_predictedAs_F_fraction,binsize=0.1,xmin=0.0,xmax=1.0)
    print(H,"\n",xe,"\n")

    print("Total num seq for validation:", len(true_seqs), len(recall))
    print("Total num seq for F_predictedAs_T:", len(F_predictedAs_T_fraction))
    print("Total num seq for T_predictedAs_F:", len(T_predictedAs_F_fraction))

    print()
    """
    
    """
    print(len(y_true_all),len(y_pred_all))
    print()
    print("Confusion Matrix:")
    print(confusion_matrix(y_true_all,y_pred_all))
    print("Recall:",recall_score(y_true_all,y_pred_all))
    print("Precision:",precision_score(y_true_all,y_pred_all))
    print("Accuracy:",accuracy_score(y_true_all,y_pred_all))
    print("MCC:",matthews_corrcoef(y_true_all,y_pred_all))
    print("AUC:",roc_auc_score(y_true_all,y_pred_all))
    print("F1 score:",f1_score(y_true_all,y_pred_all))
    """

    return {"ConfusionMatrix":confusion_matrix(y_true_all,y_pred_all),
                    "Recall":recall_score(y_true_all,y_pred_all),
                    "Precision":precision_score(y_true_all,y_pred_all),
                    "Accuracy":accuracy_score(y_true_all,y_pred_all),
                    "MCC":matthews_corrcoef(y_true_all,y_pred_all),
                    "AUC":roc_auc_score(y_true_all,y_pred_all),
                    "F1score":f1_score(y_true_all,y_pred_all)}

In [257]:
# Temporary functions
def histogram1d(data=None, binsize=None, xmin=None, xmax=None):
    if xmin is None and xmax is None:
        xmin = min(data)
        xmax = max(data)
    xedges = np.arange(xmin,xmax+binsize,binsize)
    H, xe = np.histogram(data,bins=xedges)
    #print("Value range:",xmin,xmax)
    return(H,xe)

In [263]:
def translate(seq=None,translation_dict=None):
    """
    Translates sequence of characters based on the provided translation dictionary.
    
    Parameters
    ----------
    seq              : str, default None
                       Sequence of characters to be translated
    translation_dict : dict, default None
                       Dictionary to use for translation of 'seq'.
                       All characters in the input sequence must be present in the keys of the translation dictionary.
    
    Returns
    ----------
    str
    Translated sequence
    
    """
    if translation_dict:
        return("".join([translation_dict[ch] for ch in seq]))
    else:
        return("".join([ch for ch in seq]))

    
def get_seq_dictionary(datadir=None, filelist=None, header_regex_str=None, translation_dict=None, verbose=False):
    """
    Translates sequence of characters based on the provided translation dictionary.
    
    Parameters
    ----------
    datadir          : str, default None
                       Path to directory that contains files containing sequences in FASTA format (shown below).
                       > sequence name 1
                       ALKDSFJOSDIFUSDCOSDNWEIFUCSNDJ
                       > sequence name 2
                       OSIFJODSICVJIWDCNSIDCIWF
                       > sequence name 3
                       LAKSDJFOWIEROSICJOISFJLSKDFD
    filelist         : list, default None
                       List of file names (str) to read from 'datadir'
    header_regex_str : str, default None
                       The header is parsed according to this regex string and first match group is used at the key.
                       If None then the full header line (except the last new line) is used as the key.
    translation_dict : dict, default None
                       Dictionary to use for translation of 'seq'.
                       All characters in the input sequence must be present in the keys of the translation dictionary.
    verbose          : bool, optional, default False
                       When True, prints information to STDOUT

    
    Returns
    ----------
    dict
    Dictionary of sequences.
    Keys are sequence headers read from FASTA formatted sequences.
    Values are sequences translated using 'traslation_dict'.
    If a duplicate sequence header is found then it is replaced by the one found later.
    
    """
    
    if header_regex_str is None:
        header_regex_str = '(.*)\n'

    seq = {}
    for filename in filelist:
        try:
            f = open(datadir+"/"+filename,'r')
        except:
            if verbose:
                print("Error while opening file", datadir+"/"+filename)
            continue

        for line in f.readlines():
            if line.startswith('>'):
                matchObj = re.match(header_regex_str,line,re.S)
                header = matchObj.group(1)
                seq[header] = ""
            else:
                seq[header] += translate(line.replace('\n',''),translation_dict=translation_dict)
        f.close()
    return(seq)


<h3>Define input variables</h3>

In [258]:
### Data directory
datadir = "pakTailSeqs"

In [259]:
### List of tail and full length sequence files

tail_seq_files = [
#'adk_cdhit90_tails.fasta',
#'hbha_cdhit90_tails.fasta',
#'hc1_cdhit90_tails.fasta',
#'hc2_cdhit90_tails.fasta',
'hu_cdhit90_tails.fasta'
#'ku_cdhit90_tails.fasta'
#'l22_cdhit90_tails.fasta',
#'rbr_cdhit90_tails.fasta'#? Manually identified tails are probably not correct PAK tails.
#'sig_cdhit90_tails.fasta'#? Manually identified tails are probably not correct PAK tails.
#'tig_cdhit90_tails.fasta',
#'topo1_cdhit90_tails.fasta',
#'topo3_cdhit90_tails.fasta'
]

full_seq_files = [
#'adk_cdhit90.fasta',
#'hbha_cdhit90.fasta',
#'hc1_cdhit90.fasta',
#'hc2_cdhit90.fasta',
'hu_cdhit90.fasta'
#'ku_cdhit90.fasta'
#'l22_cdhit90.fasta',
#'rbr_cdhit90.fasta'#? Manually identified tails are probably not correct PAK tails.
#'sig_cdhit90.fasta'#? Manually identified tails are probably not correct PAK tails.
#'tig_cdhit90.fasta',
#'topo1_cdhit90.fasta',
#'topo3_cdhit90.fasta'
]

In [260]:
# Define observed states for HMM
#
# Amino acid residue codes (i.e. observed states)
# Amino acids P, A, K and T are most common in manually identified PAK-tails in HU proteins.
# Below are the frequencies.
# A 0.329
# K 0.264
# T 0.184
# P 0.067
# Proline is distinctly infrequent compared to A, K and T,
# but PAK-tail analysis described by Khare et al* indicates its importance in the dynamics and conformation of the tails.
# Hence Proline is considered as a separate category.
#
# Categories: This categorization will help in discriminating P, A, K, T and other amino acids (O).
# Taking P, A, K and T in one group leads to poly-A or poly-P stretches to be labeled as PAK-tails.
# Keeping P, A, K and T as separate categories might help in learning their combinations.
# Categories are assigned as follows.
# P
# A
# K
# T
# O: OTHERS (D,E,G,A,V,L,I,M,C,N,Q,F,Y,W,P,S,T)
#
# *Conformational heterogeneity in tails of DNA-binding proteins is augmented by proline containing repeats; Khare et al 2017

# Amino acid code single letter
aa_code1 = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','X','Y']

# Amino acid categories i.e. the observed states for HMM
aa_code_cat = ['P','A','K','T','O']

# Amino acid category mapping dictionary
aa_code_cat_dict = {'P':'P',
                     'A':'A',
                     'K':'K',
                     'T':'T',
                     'C':'O', 'D':'O', 'E':'O', 'F':'O', 'G':'O', 'H':'O', 'I':'O', 'L':'O',
                     'M':'O', 'N':'O', 'Q':'O', 'R':'O', 'S':'O', 'V':'O', 'W':'O', 'X':'O', 'Y':'O'}



In [261]:
### Define hidden states for HMM. Two states here, Folded and Tail.
states = ['F','T']

In [262]:
# Number of hidden states for HMM
n_states = len(states)

# Numger of observed staets for HMM
n_states_observed = len(aa_code_cat)

<h3>Prepare input data</h3>

In [264]:
seq_tail = get_seq_dictionary(datadir=datadir, filelist=tail_seq_files, header_regex_str='>(.+).*\n', translation_dict=aa_code_cat_dict, verbose=True)
seq_full = get_seq_dictionary(datadir=datadir, filelist=full_seq_files, header_regex_str='>(.+?)\s.*\n', translation_dict=aa_code_cat_dict, verbose=True)

print("Number of tail sequences:",len(seq_tail),"\nNumber of all (full) sequences:",len(seq_full))

Number of tail sequences: 59 
Number of all (full) sequences: 1353


In [268]:
### Convert data into Pandas objects
tail = pd.Series(seq_tail)
full = pd.Series(seq_full)
print("Tail seqs:",tail.shape,"\nFull seqs:",full.shape)
print(tail.head())
#print(full.head())

Tail seqs: (59,) 
Full seqs: (1353,)
tr|A0A0T9L3R4|A0A0T9L3R4_MYCTX    AOOAPAOOOOPAAOPOAAOTOAOOPOAPAAKKAKOAKOOKOOKOOK...
tr|A0A077M8Y5|A0A077M8Y5_9MICO    AOPOKOPKTAOTOOOAPAOOATATATKAAAKKTTTAKOTTAAKKTT...
tr|W7SIK7|W7SIK7_9PSEU            OOTOKOPOOTAAAAKPAAOPAAKATAKOOATKPAOOATAAKATATO...
tr|H0K1H8|H0K1H8_9PSEU            OOAKKOPKATTTKOOTTOTOOTTAOTAKTAATOPTOTOOTTOOTOA...
tr|A0A0H5CLV3|A0A0H5CLV3_9PSEU    OOTKKOPKOAATOPAAAAPAAOATATOTTATOATTATOTTOAAAOT...
dtype: object


In [269]:
### Make dataframe containing tails and full length sequences
### Create columns containing label sequences
df = pd.concat([full, tail], axis=1, sort=True)

df.columns=['FULL','TAIL']
df = df.replace(np.nan, '', regex=True)
df['FULLLEN'] = df['FULL'].str.len()
df['TAILLEN'] = df['TAIL'].str.len()
df['FOLDLEN'] = df['FULLLEN'] - df['TAILLEN']
#df['FoldLabels'] = df['FoldLen'].apply(lambda x: 'F'*x)
#df['TailLabels'] = df['TailLen'].apply(lambda x: 'T'*x)
df['FULLLABELS'] = df['FOLDLEN'].apply(lambda x: 'F'*x) + df['TAILLEN'].apply(lambda x: 'T'*x)
#df['Tally'] = df['FullLabels'].str.len()
#df.to_csv(path_or_buf="test.csv",sep=",")

df['HEADER'] = df.index
df.reset_index(drop=True, inplace=True)

print("Columns:",df.columns)
print("Shape (all seqs):",df.shape)
#print(df[df['Tail'] != ''].head())
print("Shape (tail seqs):",df[df['TAIL'] != ''].shape)
df.head()
#df.index

Columns: Index(['FULL', 'TAIL', 'FULLLEN', 'TAILLEN', 'FOLDLEN', 'FULLLABELS',
       'HEADER'],
      dtype='object')
Shape (all seqs): (1353, 7)
Shape (tail seqs): (59, 7)


,FULL,TAIL,FULLLEN,TAILLEN,FOLDLEN,FULLLABELS,HEADER
0,OOKAOOOOOOTOKOOOOOOOATAAOOOOOOTOOOAOOKOOOOTOTO...,AOAOOOPOOOPAOKOOOATOAAKKAAOKKAPOKKAOAKKAATKAPA...,200,111,89,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,sp|O33125|DBH_MYCLE
1,OOKOOOOOAOAOKAOOTKOOAAOAOOAOOOOOOAOOKOKOOOOOAO...,,91,0,91,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,sp|P02348|DBH5_RHILE
2,OOKOOOOOAOAAOAOOPKAOAOOAOOAOOOOOTOAOKAOOOOOOOO...,,90,0,90,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,sp|P05384|DBHB_PSEAE
3,OOKOOOOOKOAAOAOOOKAAAOOAOOAOOAOOTOOOKOOOOOAOOO...,,90,0,90,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,sp|P0A1R8|DBHB_SALTY
4,OTKKOOOOOOAKKAOAKKKOOKOOOOTOOOTOTOAOAKOOKOOOOO...,,90,0,90,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,sp|P36206|DBH_THEMA


In [270]:
### Split input data into train and test sets

### Standard way of train test splitting:
### X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
### We consider amino acid sequences as X and label sqeuences as y

train_seqs, test_seqs, train_labels, test_labels = train_test_split(df.FULL, df.FULLLABELS, test_size=0.4, random_state=61)

print(train_seqs.head())
print(test_seqs.head())
print(train_seqs.shape, test_seqs.shape)
print(train_labels.shape, test_labels.shape)
#print(train_seqs.head())
#print(train_labels.head())
print("# of seqs with tails in train_seqs",sum(df.loc[train_seqs.index].TAILLEN > 0))
print("# of seqs with tails in test_seqs",sum(df.loc[test_seqs.index].TAILLEN > 0))
print("\nThe following ratios should be as similar as possible (Adjust random seed accordingly):")
print("Train seqs/tails in train seqs =",train_seqs.shape[0]/sum(df.loc[train_seqs.index].TAILLEN > 0))
print("Test seqs/tails in test seqs =",test_seqs.shape[0]/sum(df.loc[test_seqs.index].TAILLEN > 0))

668     OOKAOOOOAOAOKAOOTOOOAKKAOOAOOAOOOOAOKAOOTOOOOO...
334     OOKKOOAOOOAOOOOTOKAOOOOOOOOOOOKOOOTOTKOOOOOOAO...
17      OOKOOOOOOOATTAOOOKTOATAAOOAOOTOOOOTOAOOOOOAOTO...
1062    OOKAOOOOAOAOKOOOOOOAAOAOOOOOOAOOOAOOAOOOOOOTOO...
740     OOKOOOOOAOAAOAOOOKAOAKKAOOAOOTOOTOAOKAOOKOOOOO...
Name: FULL, dtype: object
390    OTTTPPOOOPTOOOOAAOKOOOPOKTOAOOOOOOOKOOOOAKKPOA...
789    OOTOOKOOOKKOOKTOOOAOOTOOOOOOKOAAOKTOTKAOAOKAOO...
126    OOKTOOOOOOOTKAOOOKAOAOOAOOAOOOAOTTTOKOKOTOTOOO...
542    OOKTOOOOAOAOKOOOOKKOATKAOOOOOOTOOOOOKOOOKOOOOO...
553    OOKTOOOOKOOOOTOOTKKOAOAAOOAOOOAOTOAOKOOOKOOOOO...
Name: FULL, dtype: object
(811,) (542,)
(811,) (542,)
# of seqs with tails in train_seqs 36
# of seqs with tails in test_seqs 23

The following ratios should be as similar as possible (Adjust random seed accordingly):
Train seqs/tails in train seqs = 22.52777777777778
Test seqs/tails in test seqs = 23.565217391304348


<h3>Build model on train set</h3>

In [271]:
### Build first model and write to file as json object
model_0 = train_hmm(distribution=DiscreteDistribution,
                    n_components=len(states),
                    state_names=states,
                    X=[list(s) for s in list(train_seqs)], # Needs sequences as lists of characters
                    labels=[list(s) for s in list(train_labels)], # Needs sequences as lists of characters
                    algorithm='labeled',
                    n_jobs=2)

model_0_json = model_0.to_json()
with open("model_0.json",'w') as f:
    f.write(model_0_json)

#print(model_1)
print(model_0.dense_transition_matrix())

[[9.99520333e-01 4.79667431e-04 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.00000000e-01 5.00000000e-01 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]


<h3>Model validation with grid search on transition probability matrix (Takes long time)</h3> 

In [272]:
### Grid search on transition probability matrix is needed because there is no T to F transition in this data.
### However, there could be T to F transition in new data.
### Therefore, T to F transition probability has to be non-zero.
### Grid search will allow to select a good enough model with non-zero T to F transition probability.

### Also, the selected model at this stage has to be very precise in identifying tails.
### Since tails are encoded as 1 and folded region as 0,
### the calculated precision, TP/(TP+FP), gives precision of identifying tails.
### Choose model with highest precision.

### Read model from json file

with open("model_0_PAKTO.json", 'r') as f:
    model_json = json.load(f)

for a in model_json['edges']:
    if (a[0],a[1]) == (0,0):
        FF = a[2]
    elif (a[0],a[1]) == (0,1):
        FT = a[2]
    elif (a[0],a[1]) == (1,0):
        TF = a[2]
    elif (a[0],a[1]) == (1,1):
        TT = a[2]
print(FF,FT,TF,TT)

### Set state transition probabilites for grid search
### Run this block multiple times adjusting/refining the size of the grid
FF_start = model_json['edges'][2][2]
FF_end = 0.69# 0.89
print("FF_start =",FF_start,"        FF_end =",FF_end)
TT_start = 0.999
TT_end = 0.79# 0.001
print("TT_start =",TT_start,"        TT_end =",TT_end)
step = -0.2#-0.01# 0.1
print("Step =",step)



0.9995203325694185 0.0004796674305814636 0.0 1.0
FF_start = 0.9995203325694185         FF_end = 0.69
TT_start = 0.999         TT_end = 0.79
Step = -0.2


In [304]:
### Grid search for all combinations of FF&FT and TF&TT
### Change file name for output of grid search each time
#This block takes long time to run. Uncommented line to avoid executing this block accedentally.

f = open("gridsearch_2_PAKTO.csv",'w')
f.write("FF,FT,TF,TT,Recall,Precision,Accuracy,MCC,AUC,F1score\n")
for FF in np.arange(FF_start,FF_end,step):
    FT = 1.0 - FF
    for TT in np.arange(TT_start,TT_end,step):
        TF = 1.0 - TT
        #print('{:.5f}'.format(FF), '{:.5f}'.format(FT))
        #print('{:.5f}'.format(TF), '{:.5f}'.format(TT))
        #print()
        
        #model_json['edges'][2][2] = FF
        #model_json['edges'][3][2] = FT
        #model_json['edges'][4][2] = TF
        #model_json['edges'][5][2] = TT

        cnt = 0
        for a in model_json['edges']:
            if (a[0],a[1]) == (0,0):
                model_json['edges'][cnt][2] = FF
            elif (a[0],a[1]) == (0,1):
                model_json['edges'][cnt][2] = FT
            elif (a[0],a[1]) == (1,0):
                model_json['edges'][cnt][2] = TF
            elif (a[0],a[1]) == (1,1):
                model_json['edges'][cnt][2] = TT
            cnt += 1

        model = HiddenMarkovModel.from_json(json.dumps(model_json, indent=4))
        print("Validating", cnt, len(model_json['edges']))
        #val = validate(model=model, states=states, true_seqs=test_seqs, true_labels=test_labels)
#         f.write(",".join([str(s) for s in [FF, FT, TF, TT,
#                                            val["Recall"], val["Precision"],                                            
#                                            val["Accuracy"], val["MCC"],
#                                            val["AUC"], val["F1score"]]]) + "\n")
f.close()
print(cnt)

Validating 6 6
Validating 6 6
Validating 6 6
Validating 6 6
6


In [275]:
### Generate model with selected state transition probabilities
FF = 0.9995203326
FT = 0.0004796674
TF = 0.201
TT = 0.799

with open("model_0.json", 'r') as f:
    model_json = json.load(f)

cnt = 0
for a in model_json['edges']:
    if (a[0],a[1]) == (0,0):
        model_json['edges'][cnt][2] = FF
    elif (a[0],a[1]) == (0,1):
        model_json['edges'][cnt][2] = FT
    elif (a[0],a[1]) == (1,0):
        model_json['edges'][cnt][2] = TF
    elif (a[0],a[1]) == (1,1):
        model_json['edges'][cnt][2] = TT
    cnt += 1
    
model = HiddenMarkovModel.from_json(json.dumps(model_json, indent=4))
with open("model_1.json",'w') as f:
    f.write(model.to_json())


<h3>Predict tail-like regions in new sequences using model_1</h3>

In [276]:
### Read model
model = HiddenMarkovModel.from_json('model_1.json')

In [277]:
### Get new sequences for which predictions will be generated

filelist = [
#'adk_cdhit90.fasta',
'hbha_cdhit90.fasta',
'hc1_cdhit90.fasta',
'hc2_cdhit90.fasta',
#'hu_cdhit90.fasta'
'ku_cdhit90.fasta',
#'l22_cdhit90.fasta',
#'rbr_cdhit90.fasta',#? Manually identified tails are probably not correct PAK tails.
#'sig_cdhit90.fasta',#? Manually identified tails are probably not correct PAK tails.
#'tig_cdhit90.fasta',
#'topo1_cdhit90.fasta',
#'topo3_cdhit90.fasta'
]
new_seqs = get_seq_dictionary(datadir=datadir, filelist=full_seq_files, header_regex_str='>(.+?)\s.*\n', translation_dict=aa_code_cat_dict, verbose=True)

#seq_tail = get_seq_dictionary(datadir=datadir, filelist=tail_seq_files, header_regex_str='>(.+).*\n', translation_dict=aa_code_cat_dict, verbose=True)
#seq_full = get_seq_dictionary(datadir=datadir, filelist=full_seq_files, header_regex_str='>(.+?)\s.*\n', translation_dict=aa_code_cat_dict, verbose=True)
#print("Number of tail sequences:",len(seq_tail),"\nNumber of all (full) sequences:",len(seq_full))

print("Found",len(new_seqs),"new sequences.")
#seqs = pd.Series(new_seqs)

Found 1353 new sequences.


In [286]:
### Generate dataframe from new sequences

#s = "TTFFFFTTTFFFFFTFFTTTTT"
#print(s[0:2])
df_new = pd.DataFrame(columns=['HEADER','START','END','LEN','SEQ','LABEL_SEQ','LABEL','DATASET'])

for header in new_seqs:
  seq = new_seqs[header]
  pred = model.predict(list(seq))
  pred_seq = "".join([states[i] for i in pred])
  #seq = "AABABDBADBBADBRRRRRR"
  #pred_seq = "TTFFFFTTTFFFFFTFFTTTTT"
  #print(seq)
  #print(pred_seq)

  #print(len(pred_seq))
  start = 0

  for substr in [match[0] for match in re.findall(r'((\w)\2{0,})', pred_seq)]:
    ### This regex also works: [match[0] for match in re.findall(r'((.)\2{,})', pred_seq)]
    #print(substr)
    end = start+len(substr)
    #print(start,end)
    #print(header,start,end,seq[start:end],pred_seq[start:end],'NEW_SEQS')
#     df_new = df_new.append(pd.DataFrame([(header, start, end, end-start,
#                                                   seq[start:end], pred_seq[start:end],
#                                                   pred_seq[start], 'NEW_SEQS')],
#                                                 columns=['HEADER','START','END','LEN',
#                                                          'SEQ','LABEL_SEQ','LABEL','DATASET']),
#                                    ignore_index=True)

    df_new = pd.concat([df_new,
                        pd.DataFrame([(header, start, end, end-start,
                                       seq[start:end], pred_seq[start:end],
                                       pred_seq[start], 'NEW_SEQS')],
                                       columns=['HEADER','START','END','LEN','SEQ','LABEL_SEQ','LABEL','DATASET']
                                    )
                       ],
                       ignore_index=True)

    start += len(substr)
  #print(header,start,end,seq[start:end],pred_seq[start:],'NEW_SEQS')
  #break

print(df.shape)
df_new.head()
#new_seq_df[new_seq_df.LABEL == 'F']['LEN'].plot.hist()

(1353, 7)


,HEADER,START,END,LEN,SEQ,LABEL_SEQ,LABEL,DATASET
0,tr|A0A064BZG4|A0A064BZG4_STREE,0,91,91,OAOKOOOOAKOAOATOOTKKOOAAAOOAOOAAOAOOOAAOOKOOOO...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,F,NEW_SEQS
1,sp|O33125|DBH_MYCLE,0,109,109,OOKAOOOOOOTOKOOOOOOOATAAOOOOOOTOOOAOOKOOOOTOTO...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,F,NEW_SEQS
2,sp|O33125|DBH_MYCLE,109,194,85,AAKKAAOKKAPOKKAOAKKAATKAPAKKAOKAPAKKOTTAOKOPAK...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,T,NEW_SEQS
3,sp|O33125|DBH_MYCLE,194,200,6,AKOOOK,FFFFFF,F,NEW_SEQS
4,sp|P0A1R8|DBHB_SALTY,0,90,90,OOKOOOOOKOAAOAOOOKAAAOOAOOAOOAOOTOOOKOOOOOAOOO...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,F,NEW_SEQS


In [288]:
### ADD a column to train/test data (i.e. Dataframe df) to hold dataset name
df['DATASET'] = ''
df['DATASET'].iloc[train_seqs.index] = 'TRAIN'
df['DATASET'].iloc[test_seqs.index] = 'TEST'
#print(df['DATASET'].value_counts())
#print(train_seqs.shape,test_seqs.shape)
#print(df[df['DATASET']=='TRAIN'].head())
print("Columns in df:",df.columns)
print(df['DATASET'].value_counts())

### Create temporary dataframe in required format
df_train_test = pd.DataFrame(columns=['HEADER','START','END','LEN','SEQ','LABEL_SEQ','LABEL','DATASET'])

for i in df.index:
  #print(df.iloc[i])
  row = df.iloc[i]
  start = 0
  for substr in [match[0] for match in re.findall(r'((\w)\2{0,})', row['FULLLABELS'])]:
    #print("*",substr)
    end = start+len(substr)
    #print(start,end)
#     df_train_test = df_train_test.append(pd.DataFrame([(row['HEADER'], start, end, end-start,
#                                                   row['FULL'][start:end], row['FULLLABELS'][start:end],
#                                                   row['FULLLABELS'][start], row['DATASET'])],
#                                                 columns=['HEADER','START','END','LEN',
#                                                          'SEQ','LABEL_SEQ','LABEL','DATASET']),
#                                    ignore_index=True)

    df_train_test = pd.concat([df_train_test,
                               pd.DataFrame([(row['HEADER'], start, end, end-start,
                                              row['FULL'][start:end], row['FULLLABELS'][start:end],
                                              row['FULLLABELS'][start], row['DATASET'])],
                                              columns=['HEADER','START','END','LEN','SEQ','LABEL_SEQ','LABEL','DATASET']
                                           )
                              ],
                              ignore_index=True)

    start += len(substr)
  
df_train_test.head()

<ipython-input-288-4536cbf966ad>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATASET'].iloc[train_seqs.index] = 'TRAIN'
<ipython-input-288-4536cbf966ad>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATASET'].iloc[test_seqs.index] = 'TEST'


Columns in df: Index(['FULL', 'TAIL', 'FULLLEN', 'TAILLEN', 'FOLDLEN', 'FULLLABELS', 'HEADER',
       'DATASET'],
      dtype='object')
TRAIN    811
TEST     542
Name: DATASET, dtype: int64


,HEADER,START,END,LEN,SEQ,LABEL_SEQ,LABEL,DATASET
0,sp|O33125|DBH_MYCLE,0,89,89,OOKAOOOOOOTOKOOOOOOOATAAOOOOOOTOOOAOOKOOOOTOTO...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,F,TEST
1,sp|O33125|DBH_MYCLE,89,200,111,AOAOOOPOOOPAOKOOOATOAAKKAAOKKAPOKKAOAKKAATKAPA...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,T,TEST
2,sp|P02348|DBH5_RHILE,0,91,91,OOKOOOOOAOAOKAOOTKOOAAOAOOAOOOOOOAOOKOKOOOOOAO...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,F,TEST
3,sp|P05384|DBHB_PSEAE,0,90,90,OOKOOOOOAOAAOAOOPKAOAOOAOOAOOOOOTOAOKAOOOOOOOO...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,F,TEST
4,sp|P0A1R8|DBHB_SALTY,0,90,90,OOKOOOOOKOAAOAOOOKAAAOOAOOAOOAOOTOOOKOOOOOAOOO...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,F,TEST


In [289]:
### Concatanate dataframes for new sequences and train/test dataset
print(df_train_test.columns)
print(df_new.columns)

df_2 = pd.concat([df_train_test, df_new], axis=0, ignore_index=True)

print(df_2['DATASET'].value_counts())
df_2.head()

Index(['HEADER', 'START', 'END', 'LEN', 'SEQ', 'LABEL_SEQ', 'LABEL',
       'DATASET'],
      dtype='object')
Index(['HEADER', 'START', 'END', 'LEN', 'SEQ', 'LABEL_SEQ', 'LABEL',
       'DATASET'],
      dtype='object')
NEW_SEQS    1465
TRAIN        847
TEST         565
Name: DATASET, dtype: int64


,HEADER,START,END,LEN,SEQ,LABEL_SEQ,LABEL,DATASET
0,sp|O33125|DBH_MYCLE,0,89,89,OOKAOOOOOOTOKOOOOOOOATAAOOOOOOTOOOAOOKOOOOTOTO...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,F,TEST
1,sp|O33125|DBH_MYCLE,89,200,111,AOAOOOPOOOPAOKOOOATOAAKKAAOKKAPOKKAOAKKAATKAPA...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,T,TEST
2,sp|P02348|DBH5_RHILE,0,91,91,OOKOOOOOAOAOKAOOTKOOAAOAOOAOOOOOOAOOKOKOOOOOAO...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,F,TEST
3,sp|P05384|DBHB_PSEAE,0,90,90,OOKOOOOOAOAAOAOOPKAOAOOAOOAOOOOOTOAOKAOOOOOOOO...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,F,TEST
4,sp|P0A1R8|DBHB_SALTY,0,90,90,OOKOOOOOKOAAOAOOOKAAAOOAOOAOOAOOTOOOKOOOOOAOOO...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,F,TEST
